<a href="https://colab.research.google.com/github/giovannasantoloni/Projetos-de-Sistemas-Inteligentes-de-Apoio-Decis-o/blob/main/Projeto_Final_Curso_BI_Master_2021_1_Giovanna_Dohnert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Final Curso BI Master 2021-1
> Por Giovanna Santoloni Dohnert

In [1]:
#Montagem do Drive 
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#Importar Bibliotecas
import numpy as np
import pandas as pd
import os
import shutil
import cv2
from glob import glob
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
import random
import shutil


In [3]:
#Importar Aplicações
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from scipy import ndimage, misc
import skimage
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Sequential
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models.cloning import training
from tensorflow.keras.layers import Dense, Dropout


In [4]:
#Caminho da Base de Dados
import os
workdir_path = '/content/drive/My Drive/Projeto Final BI/images'  # Local da pasta onde estão os arquivos de entrada (full dataset)
os.chdir(workdir_path)


In [5]:
# Caminho da pasta que contém todas as imagens
full_dataset = r'/content/drive/My Drive/Projeto Final BI/images'

In [6]:
#Definição de Parâmetros

#Número de Classes 
n_classes = 9

#Tamanho das Imagens
target_size = (280, 280)

#Tamanho do Lote para Treinamento
batch_size = 64


In [7]:
# Separando o Conjunto de Dados em Treino e Validação

image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2, rotation_range=20, # aumentando a variabilidade de dados
width_shift_range=0.2,
height_shift_range=0.2,
zoom_range=0.2,
horizontal_flip=True
)

train_dataset = image_generator.flow_from_directory(batch_size=batch_size, 
                                                    directory=full_dataset ,
                                                    shuffle = True,
                                                    target_size =target_size,
                                                    subset ="training",
                                                    class_mode = 'categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=batch_size, 
                                                    directory=full_dataset,
                                                    shuffle = True,
                                                    target_size =target_size,
                                                    subset ="validation",
                                                    class_mode = 'categorical')

Found 4239 images belonging to 9 classes.
Found 1055 images belonging to 9 classes.


In [8]:
# Separando o conjunto de Teste
image_generator = ImageDataGenerator(rescale=1./255)

test_dataset = image_generator.flow_from_directory(batch_size=batch_size, 
                                                    directory=full_dataset,
                                                    shuffle = True,
                                                    target_size =target_size,
                                                    class_mode = 'categorical')

Found 5294 images belonging to 9 classes.


In [9]:
# carregar a arquitetura VGG16 pré-treinada
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(*target_size, 3))

58889256/58889256 [==============================] - 4s 0us/step


In [10]:
# adicionar camadas adicionais ao modelo
x = base_model.output
x = Flatten()(x)
x = Dropout(0.5)(x) # aumentando o dropout para evitar overfitting
predictions = Dense(n_classes, activation='softmax')(x)

In [11]:
# criar modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# congelar as camadas da base
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# compilar o modelo
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# Treinar o modelo
history = model.fit(
    train_dataset,
    steps_per_epoch=len(train_dataset),
    epochs=10,
    validation_data=validation_dataset,
    validation_steps=len(validation_dataset))

Epoch 1/10
 3/67 [>.............................] - ETA: 39:52 - loss: 2.2732 - acc: 0.1927